In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [2]:
def circuit(circuit_param):
    qml.RY(circuit_param, wires=0)
    qml.Hadamard(wires=0)
    qml.T(wires=0)

In [3]:
def state_purity(angle, phase, circuit_param, noise_param):

    def noise(Beta):
        qml.CRX(Beta ,wires=(0,1))
        qml.CNOT(wires=(1,0))
        
    dev = qml.device("default.mixed", wires=2)
    @qml.qnode(dev)
    
    def noisy_circuit(angle, phase, circuit_param, noise_param):

        phi = phase 
        theta = (-2) * angle
        omega = (-1) * phase
        alpha = circuit_param
        beta = noise_param #parameters
        
        qml.Rot(phi, theta, omega, wires=0) #state has been prepared 
        
        qml.RY(alpha, wires=0)
        noise(beta)
        qml.Hadamard(wires=0)
        noise(beta)
        qml.T(wires=0)
        noise(beta)
        
        r_x = qml.expval(qml.PauliX(0)) #expectation value
        r_y = qml.expval(qml.PauliY(0))
        r_z = qml.expval(qml.PauliZ(0))
        
        return (r_x,r_y,r_z)
    
    R = noisy_circuit(angle, phase, circuit_param, noise_param)
    R_2 = (R ** 2) 
    A_2 = sum(R_2)
    Purity = (1 + A_2) / 2
    
    return  Purity

In [4]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    output = state_purity(*ins)

    return str(np.real_if_close(output))

def check(solution_output: str, expected_output: str) -> None:
    """
    Compare solution with expected.

    Args:
            solution_output: The output from an evaluated solution. Will be
            the same type as returned.
            expected_output: The correct result for the test case.

    Raises:
            ``AssertionError`` if the solution output is incorrect in any way.
    """

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-2
    ), "Your calculated purity isn't quite right."

In [5]:
test_cases = [['[0.1,0.2,0.3,0.4]', '0.9647'], ['[0.5,0.3,0.5,0.7]', '0.928356']]

In [6]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.1,0.2,0.3,0.4]'...
Correct!
Running test case 1 with input '[0.5,0.3,0.5,0.7]'...
Correct!
